In [1]:
import numpy as np
import graph_tool.all as gt

Unable to init server: Could not connect: Connection refused
Unable to init server: Could not connect: Connection refused


In [2]:
L = 20
J1 = 1
ratio = -0.9
J2 = ratio * np.abs(J1)
g = gt.lattice([L, L], periodic=False)

In [3]:
weights = {}

edges = [(*edge, J1) for edge in g.get_edges().tolist()]

indices = g.get_vertices().reshape([L, L])
for x in range(L-1):
    for y in range(L-1):
        # diagonals on right
        edges.append((*indices[[x, x+1], [y, y+1]], J2))
        edges.append((*indices[[x+1, x], [y, y+1]], J2))

In [4]:
pos = []
for x in range(L):
    for y in range(L):
        pos.append([x, y])

pos = np.array(pos)

In [88]:
epochs = 2000
n_errors = 100
g = gt.Graph(directed=False)
weights = g.new_ep("double")

g.add_edge_list(edges, eprops=[weights])
pos = g.new_vertex_property("vector<double>", pos)

T = 0.5
Mag = np.zeros(n_errors)
X = np.zeros(n_errors)
for i in range(n_errors):
    M = []
    state = gt.IsingMetropolisState(g, beta=1/T, w=weights)
    state.iterate_async(niter=1000)
    for e in range(epochs):
        state.iterate_async(niter=g.num_vertices())
        M.append(np.abs(state.s.fa.mean()))
        
    Mag[i] = np.mean(M)
    X[i] = np.var(M)

In [92]:
Mag.mean(), Mag.std()

(0.023935650000000006, 0.02208876724044373)

In [91]:
X.mean()

0.00035937727087500003